<a href="https://colab.research.google.com/github/dariashcherbakovaaa/MA_DSE_Unimi/blob/network_science/Network_science_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
import itertools

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import learning_curve, validation_curve, train_test_split, KFold, StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV, cross_validate, RepeatedStratifiedKFold
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier

from scipy.stats import loguniform, beta, uniform

from mlxtend.feature_selection import SequentialFeatureSelector as SFS

from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.pipeline import Pipeline as IMBPipeline

import missingno as msno

import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

import networkx as nx


In [11]:
data = pd.read_excel('data.xlsx')
data.sample(10)

,Name,Supervisor Name,Title,Age,Sex,Education,WorkTitleYear,WorkTitleMonth,WorkOrgYear,WorkOrgMonth,...,QS27R,QS28,QS29,QS30,QS31R,QS32,QS33,QS34,QS35R,QS36
40,MAYu,DDI,Specialist,22,1,3,0,3,0,6,...,6.0,4.0,7.0,5.0,7.0,1.0,2.0,1.0,7.0,5.0
42,MKC,VCI,Specialist,25,1,3,1,0,1,0,...,3.0,2.0,2.0,3.0,3.0,4.0,3.0,6.0,6.0,4.0
51,PNYu,DDI,Project manager,28,1,3,0,6,0,6,...,6.0,4.0,4.0,4.0,7.0,1.0,2.0,1.0,7.0,3.0
38,KNN1,TIG,Lead specialist,43,1,3,6,0,9,0,...,7.0,5.0,4.0,4.0,7.0,1.0,1.0,1.0,7.0,5.0
52,RPM,ADN,Lawyer,25,0,3,3,0,3,0,...,2.0,2.0,2.0,2.0,3.0,2.0,6.0,6.0,3.0,3.0
16,CMC,LDA,Subsidiary manager,37,1,3,5,6,5,6,...,7.0,7.0,5.0,6.0,6.0,2.0,1.0,3.0,6.0,6.0
65,ZAR,TIG,Lead specialist,30,1,3,3,0,7,0,...,5.0,3.0,1.0,2.0,4.0,5.0,1.0,6.0,5.0,1.0
28,GOV,MAN,Lead specialist,29,0,3,1,0,3,0,...,5.0,3.0,2.0,2.0,4.0,3.0,2.0,3.0,5.0,4.0
66,ZhAV,VEYu,Assistant department head,26,1,3,0,6,0,6,...,6.0,4.0,4.0,4.0,7.0,1.0,2.0,1.0,7.0,3.0
34,KIM,TAA,Lead specialist,28,1,3,6,0,6,0,...,6.0,6.0,6.0,6.0,5.0,2.0,1.0,3.0,6.0,6.0


In [12]:
data.shape

(68, 87)

In [16]:
from skimpy import skim

In [21]:
skim(data)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 68     │ │ int64       │ 49    │                                                          │
│ │ Number of columns │ 87     │ │ float64     │ 35    │                                                          │
│ └───────────────────┴────────┘ │ string      │ 3     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name           ┃ NA   ┃ NA %    ┃ mean   ┃ sd     ┃ p0   ┃ p25   ┃ p50   ┃ p75   ┃ p100   ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━┩  │
│ │ Age                   │    0 │       0 │     30 │    6.7 │   20 │    25 │    28 │    35 │     51 │ ▇▇▅▃▁▁  │  │
│ │ Sex                   │    0 │       0 │   0.74 │   0.44 │    0 │     0 │     1 │     1 │      1 │ ▃    ▇  │  │
│ │ Education             │    0 │       0 │      3 │   0.21 │    2 │     3 │     3 │     3 │      4 │     ▇   │  │
│ │ WorkTitleYear         │    0 │       0 │    2.3 │      2 │    0 │     1 │   1.5 │     4 │      7 │ ▇▂▂▁▁▂  │  │
│ │ WorkTitleMonth        │    0 │       0 │    2.5 │    3.4 │    0 │     0 │     0 │     6 │     10 │ ▇▁ ▃▁▁  │  │
│ │ WorkOrgYear           │    0 │       0 │    3.2 │    2.7 │    0 │     1 │   2.5 │     5 │      9 │ ▇▂▃▂▃▁  │  │
│ │ WorkOrgMonth          │    0 │       0 │    2.4 │    3.6 │    0 │     0 │     0 │     6 │     11 │ ▇▁ ▂▁▁  │  │
│ │ RepSupYear            │    0 │       0 │    2.3 │    2.1 │    0 │     1 │     1 │     4 │      7 │ ▇▁▂▁▁▂  │  │
│ │ RepSupMonth           │    0 │       0 │    2.4 │    3.6 │    0 │     0 │     0 │     6 │     11 │ ▇  ▂▁▁  │  │
│ │ RespNumber            │    0 │       0 │     34 │     20 │    1 │    18 │    34 │    51 │     68 │ ▇▇▇▇▇▇  │  │
│ │ NumInd                │    0 │       0 │    6.6 │    2.7 │    2 │     5 │     6 │     8 │     12 │ ▂▇▅▇▁▃  │  │
│ │ Q1                    │    0 │       0 │      6 │    1.1 │    3 │     5 │     6 │     7 │      7 │ ▁▁ ▃▇▇  │  │
│ │ Q2                    │    0 │       0 │    5.9 │    1.4 │    1 │     5 │     6 │     7 │      7 │   ▁▁▁▇  │  │
│ │ Q3                    │    0 │       0 │    5.7 │    1.5 │    1 │     5 │     6 │     7 │      7 │   ▁▁▂▇  │  │
│ │ Q4                    │    0 │       0 │    4.1 │    2.2 │    1 │     2 │   3.5 │     6 │      7 │ ▃▅▅ ▃▇  │  │
│ │ Q5                    │    0 │       0 │      4 │    2.2 │    1 │     2 │   3.5 │     6 │      7 │ ▃▅▃▁▃▇  │  │
│ │ Q6                    │    0 │       0 │    3.7 │    2.2 │    1 │     2 │     3 │     6 │      7 │ ▆▅▅▃▂▇  │  │
│ │ Q7                    │    0 │       0 │    5.8 │    1.2 │    2 │     5 │     6 │     7 │      7 │  ▁▁▃▇▅  │  │
│ │ Q8                    │    0 │       0 │    5.6 │    1.3 │    2 │     5 │     6 │     7 │      7 │  ▃▁▅▇▆  │  │
│ │ Q9                    │    0 │       0 │    5.5 │    1.5 │    2 │   4.8 │     6 │     7 │      7 │  ▃▂▃▅▇  │  │
│ │ Q10                   │    0 │       0 │    5.5 │    1.5 │    2 │     5 │     6 │     7 │      7 │ ▁▂▁▆▅▇  │  │
│ │ Q11                   │    0 │       0 │    5.5 │   

In [27]:
data['Education'].value_counts() # not representative to use in the analysis

3    65
4     2
2     1
Name: Education, dtype: int64

In [28]:
data['WorkOrgYear']

0     1
1     7
2     8
3     2
4     2
     ..
63    1
64    7
65    7
66    0
67    1
Name: WorkOrgYear, Length: 68, dtype: int64

In [32]:
support = pd.read_excel("data.xlsx", sheet_name='Support')
support.head(5)

,Unnamed: 0,YuEYu,PNA,BNV,GGV,GEB,CEB,DDN,ZhAV,VEYu,...,ZNE,BVZ1,PIV1,BYuV,KAA2,CDV,BMYu1,GAK1,AAA1,MRC1
0,YuEYu,NaN,7.0,7.0,5.0,5.0,5.0,5.0,6.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PNA,5.0,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BNV,NaN,NaN,NaN,2.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GGV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GEB,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
